This is a hacky script to get the timing data from the Psychopy scripts into AFNI-friendly format
for TechScan_900. I can make this a bit less hacky, but the big take-home is that changes to logging in
PsychoPy will make this a lot easier. Right now the start times for the audio trials are not logged.
Also, the keypresses are not logged in the csv file. 

In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [5]:
sbjid = 'sub-02' # Hacky for now. Will need to change for each participant
sbjnum = '2'
Rootdir = f"/Volumes/NIMH_SFIM/handwerkerd/ComplexMultiEcho1/Data/{sbjid}/DataOffScanner/psychopy/"

Runnums = [4, 5, 6]
os.chdir(Rootdir)

In [6]:
ProcTypes = np.array(['VisProc', 'FalVisProc', 'AudProc', 'FalAudProc'])
ShowExpected=True # Adds more print lines to show measured vs expected timings
NumTrials=12 # There should be 12 trials per condition
for ProcType in ProcTypes:
    TrialTimes = np.empty((NumTrials, len(Runnums)))
    for runidx in range(3):
        Runnum = Runnums[runidx]
        csvfilenames = glob.glob(f"{sbjnum}_WordNonword_fastloc_RUN{Runnum}_????_???_??_????.csv")
        if len(csvfilenames)>1:
            print(f"Error: More than one file: {csvfilenames}")
        else:
            rundata = pd.read_csv(csvfilenames[0])
            # For each trial type load the measured time the first stimulus appeared in a trial (stim1_time)
            #   and when that stimulus was supposed to apper (StartSec)
            TrialTimes[:,runidx] = np.around(rundata.loc[rundata['Procedure']==ProcType,'stim1_time'].values,decimals=1)
            ExpectedStart = np.around(rundata.loc[rundata['Procedure']==ProcType,'StartSec'].values,decimals=1)
            if ShowExpected:
                print(f"Run {Runnum} {ProcType}")
                print(f"Actual Times:   {TrialTimes[:,runidx]}")
                print(f"Expected Times: {ExpectedStart}")
    # Save a separate .1D file for each trial type
    np.savetxt(f"{ProcType}_Times.1D", TrialTimes.T, fmt='%4.1f')



Run 4 VisProc
Actual Times:   [ 37.5  99.  118.5 148.5 169.5 199.5 228.  331.5 444.  454.5 465.  474. ]
Expected Times: [ 37.5  99.  118.5 148.5 169.5 199.5 228.  331.5 444.  454.5 465.  474. ]
Run 5 VisProc
Actual Times:   [ 54.   76.5  87.  132.  150.  189.  198.  319.5 360.  412.5 421.5 483. ]
Expected Times: [ 54.   76.5  87.  132.  150.  189.  198.  319.5 360.  412.5 421.5 483. ]
Run 6 VisProc
Actual Times:   [ 36.   54.   82.5 115.5 198.  207.  225.  235.5 337.5 390.  400.5 469.5]
Expected Times: [ 36.   54.   82.5 115.5 198.  207.  225.  235.5 337.5 390.  400.5 469.5]
Run 4 FalVisProc
Actual Times:   [ 13.5  25.5  81.   90.  108.  129.  249.  307.5 348.  405.  435.  493.5]
Expected Times: [ 13.5  25.5  81.   90.  108.  129.  249.  307.5 348.  405.  435.  493.5]
Run 5 FalVisProc
Actual Times:   [ 22.5  66.  220.5 229.5 270.  280.5 291.  328.5 351.  369.  462.  472.5]
Expected Times: [ 22.5  66.  220.5 229.5 270.  280.5 291.  328.5 351.  369.  462.  472.5]
Run 6 FalVisProc
Actual 

In [7]:
# Any of the following 4 buttons is considered a response
ButtonKeys = ['Keypress: r', 'Keypress: b', 'Keypress: g', 'Keypress: y']
KeypressTimesAllRuns = []
for runidx in range(3):
    triggertimes = np.empty(1000) # timestamps the trigger 't' was pressed
    keypresstimes = np.empty(1000) # timestamps a button was pressed
    ExpectedResponseTimes = np.empty(1000) # When the 4th stimulus appeared in a trial (after which a response might be expected)
    Runnum = Runnums[runidx]
    # The 4th stimulus times are only in the csv file and the other times are in the log file
    csvfilenames = glob.glob(f"{sbjnum}_WordNonword_fastloc_RUN{Runnum}_????_???_??_????.csv")
    if len(csvfilenames)>1:
        print(f"Error: More than one file: {csvfilenames}")
    else:
        rundata = pd.read_csv(csvfilenames[0])
        logfilenames = glob.glob(f"{sbjnum}_WordNonword_fastloc_RUN{Runnum}_????_???_??_????.log")
        logdata = pd.read_csv(logfilenames[0], sep='\t')

    # Find the times when the 4th stimulus appeared for each trial
    Stim4Times = rundata.loc[rundata['Procedure'] != 'Null','stim4_time'].values
    Stim4Times = Stim4Times[~np.isnan(Stim4Times)]

    logvals = logdata.iloc[:,2].values
    trigidx = 0
    keypressidx = 0
    ntidx = 0
    for lidx in range(len(logvals)):
        # For the log lines that initialize each trial, if there's
        # "('Expected_Response', 1)" then a button press is expected
        # Identify those trials and log their trial number "TrialN"
        if 'New trial (' in logvals[lidx]:
            tmp = logdata.iloc[lidx,2]
            tmpidx = np.char.find(tmp, 'Expected_Response')
            tmpbool = tmp[tmpidx+20]
            if tmpbool == '1':
                Rtrialstartidx = np.char.find(tmp,'TrialN')
                Rtrialendidx = np.char.find(tmp, "), ('ProcedureID'")
                tmpTrialNumber = float(tmp[(Rtrialstartidx+9):Rtrialendidx])
                print(tmpTrialNumber)
                ExpectedResponseTimes[ntidx] = Stim4Times[int(tmpTrialNumber-1)]
                ntidx+=1

        if 'Keypress: t' in logvals[lidx]:
            triggertimes[trigidx] = logdata.iloc[lidx,0]
            trigidx+=1
        if any(x in logvals[lidx] for x in ButtonKeys):
            keypresstimes[keypressidx] = logdata.iloc[lidx,0]
            keypressidx+=1

    # Truncate the hacky extra long arrays I stated with
    ExpectedResponseTimes = ExpectedResponseTimes[:ntidx]
    triggertimes = triggertimes[:trigidx]
    # The difference between every 2 trigger pulses shoudl be the TR (with error form keyboard sampling variation)
    VolSpacing = triggertimes[1:]-triggertimes[:-1]

    # keypresstimes are a global clock while the STime4Times are initialized
    # by the first TR. Subtract the first trigger time so they are on the same clock
    keypresstimes = keypresstimes[:keypressidx]-triggertimes[0]

    # Go through all trials where a response is expected and count trials with or without a response
    MissedResponseCount = 0
    CorrectResponseCount = 0
    for eidx in range(len(ExpectedResponseTimes)):
        tmp = np.any((keypresstimes >= (ExpectedResponseTimes[eidx]-0.5)) & (keypresstimes <= (3+ExpectedResponseTimes[eidx])))
        if tmp:
            CorrectResponseCount += 1
        else:
            MissedResponseCount += 1

    # print(f"Keypress Times {keypresstimes}")
    # print(f"Expected Response Times {ExpectedResponseTimes}")

    # Go through all keypresses and log the key presses that are correct or incorrect
    # Note: For now, ALL key presses are given to the GLM in one regressor, but
    #   tracking correct and incorrect responses is useful to gauge attention level
    CorrectKeypressTimes = np.empty(len(keypresstimes))
    IncorrectKeypressTimes = np.empty(len(keypresstimes))
    cidx=0
    iidx=0
    for kidx in range(len(keypresstimes)):
        tmp = np.any(((ExpectedResponseTimes-0.25)<=keypresstimes[kidx]) & (ExpectedResponseTimes>=(keypresstimes[kidx]-3)))
        if tmp:
            CorrectKeypressTimes[cidx] = keypresstimes[kidx]
            cidx += 1
        else:
            IncorrectKeypressTimes[iidx] = keypresstimes[kidx]
            iidx += 1
    CorrectKeypressTimes = CorrectKeypressTimes[:cidx]
    IncorrectKeypressTimes = IncorrectKeypressTimes[:iidx]
    print(f"VolumeSpacing (TR): mean {np.mean(VolSpacing)} std {np.std(VolSpacing)} absmax {np.max(np.abs(VolSpacing-1.5))}")
    print(f"Correct Keypress Times: {CorrectKeypressTimes}")
    print(f"Incorrect Keypress Times: {IncorrectKeypressTimes}")
    print(f"Correct, Missed, Incorrect Keypress Count: {CorrectResponseCount} {MissedResponseCount} {len(IncorrectKeypressTimes)}")
    KeypressTimesAllRuns.append(keypresstimes)

# Save all the keypress times
with open("Keypress_Times.1D", mode='a') as keyfile:
    for runidx in range(len(Runnums)):
        np.savetxt(keyfile, KeypressTimesAllRuns[runidx], fmt='%4.1f', newline="\t")
        np.savetxt(keyfile, [''], fmt='%s')



10.0
13.0
19.0
24.0
26.0
27.0
35.0
42.0
VolumeSpacing (TR): mean 1.4999492795389051 std 0.00587520244789272 absmax 0.018000000000029104
Correct Keypress Times: [111.6655 141.7818 203.2798 252.6964 270.8628 279.9462 377.3945 447.827 ]
Incorrect Keypress Times: [ 50.2153 262.3461 399.8779 487.7428]
Correct, Missed, Incorrect Keypress Count: 8 0 4
12.0
13.0
16.0
19.0
20.0
32.0
36.0
48.0
VolumeSpacing (TR): mean 1.4999907780979826 std 0.003495394454376228 absmax 0.017899999999997362
Correct Keypress Times: [135.7459 144.9461 175.0461 201.7287 211.1284 332.4437 372.9098 506.5433]
Incorrect Keypress Times: [ 37.1645 184.0458 254.8784 453.9424]
Correct, Missed, Incorrect Keypress Count: 8 0 4
3.0
8.0
13.0
15.0
16.0
31.0
34.0
47.0
VolumeSpacing (TR): mean 1.499942074927954 std 0.00528495927533298 absmax 0.01830000000001064
Correct Keypress Times: [ 39.8319  86.3316 144.7644 166.0299 175.3137 324.712  362.2447 495.8599]
Incorrect Keypress Times: [28.7324]
Correct, Missed, Incorrect Keypress Cou